In [1]:
import torch
import numpy as np
import transforms_data as td
from PIL import Image
import glob
from torch import cuda


CATEGORIES = ["Falco", "Jesse", "Konrad", "Nils", "Johannes"]
CATEGORIES.sort()
AUTHORIZED = ["Falco","Konrad"]
RESIZE_Y = 150
RESIZE_X = 100
DATA_TEST_FOLDER = "../test/*png"

model = torch.load('./class_sample/model.pt')

In [2]:
image_array = []
files = glob.glob (DATA_TEST_FOLDER)
files.sort()
# declare function of transformation
preprocess = td.transforms_valid_and_test((RESIZE_Y, RESIZE_X),[0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

for f in files:
    image = Image.open(f)
    image_transformed = preprocess(image)
    image_transformed = image_transformed.reshape(-1, RESIZE_Y, RESIZE_X, 1)
    image_transformed = image_transformed.permute(3, 0, 1, 2)
    if cuda.is_available():
        image_array.append(image_transformed.type('torch.cuda.FloatTensor'))
    else:
        image_array.append(image_transformed.type('torch.FloatTensor'))

In [3]:
all_classes = []
summ_pred = np.empty(1)
for i in image_array:
	with torch.no_grad():
		pred = model(i)
		pred = torch.softmax(pred, 1)
		pred = pred.cpu().numpy()
		summ_pred = summ_pred + pred

	classes = np.argmax(pred, 1)
	all_classes.append(classes[0])

	pred = np.append(pred, classes)
	pred = np.append(pred, CATEGORIES[classes[0]])	
	print(pred, "\n")
print(all_classes)
print(summ_pred)


['0.19533786177635193' '0.04059185832738876' '0.5637624263763428'
 '0.02813841961324215' '0.17216938734054565' '2.0' 'Johannes'] 

['0.789467990398407' '0.031321123242378235' '0.035172224044799805'
 '0.038966841995716095' '0.10507182776927948' '0.0' 'Falco'] 

['0.6517340540885925' '0.04698029160499573' '0.06975772231817245'
 '0.07839997857809067' '0.15312793850898743' '0.0' 'Falco'] 

['0.7030173540115356' '0.25509652495384216' '0.0046354029327631'
 '0.010346267372369766' '0.02690448798239231' '0.0' 'Falco'] 

['0.8828981518745422' '0.02076706290245056' '0.008723532781004906'
 '0.04275737330317497' '0.04485388845205307' '0.0' 'Falco'] 

['0.9697948098182678' '0.021453814581036568' '0.00019798381254076958'
 '0.0008644312620162964' '0.0076888869516551495' '0.0' 'Falco'] 

['0.8816684484481812' '0.07324919104576111' '0.0010174292838200927'
 '0.003983926959335804' '0.040081024169921875' '0.0' 'Falco'] 

['0.8315247297286987' '0.04502866417169571' '0.0008458319352939725'
 '0.00120946962852

In [4]:
# Hier besser die Warscheinlichkeit über alle Bilder ermitteln und darüber prüfen.
# Beispiel: Bei 5 Bilder muss die aufsummierte Wahrscheinlichkeit für eine Person >4 sein!!

NUMBER_AUTHORIZED = int(.8*len(image_array))
authentification_dict = {CATEGORIES[i]:all_classes.count(i) for i in all_classes}
print(authentification_dict) 

for a in authentification_dict:
    if a in AUTHORIZED and authentification_dict[a] >= NUMBER_AUTHORIZED:
        print("Access granted! Welcome "  + a + "!")
    else:
        print("Access denied")
        break


{'Johannes': 1, 'Falco': 8}
Access denied


In [ ]:
PERCENTAGE_AUTHORIZED = 8
authentification_dict = {CATEGORIES[i]:all_classes.count(i) for i in all_classes}
print(authentification_dict) 

for a in authentification_dict:

    




    
